In [1]:
% load_ext autoreload
% autoreload 2
import pandas as pd
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from random import shuffle, seed

/Users/Louis/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Task 1 : Linear Models

### Preprocessing

In [2]:
seed(1)

In [3]:
df_2015 = pd.read_csv('dataset/2015.csv', sep=",")
df_2016 = pd.read_csv('dataset/2016.csv', sep=",")
df_2017 = pd.read_csv('dataset/2017.csv', sep=",")
df_2018 = pd.read_csv('dataset/2018.csv', sep=",")

df_list = [df_2015, df_2016, df_2017, df_2018]

In [4]:
def get_dico(df_to_clean, p_column=100, p_row=100):
    df = df_to_clean.copy()
    dico_column = (df.isnull().sum()/np.shape(df)[0]*100).to_dict()
    dico_row = (df.isnull().sum(axis=1)/len(df.columns)*100).to_dict()
    for key, value in dico_column.items():
        if value >= p_column:
            del df[key]
    for key, value in dico_row.items():
        if value >= p_row:
            df.drop(key, inplace=True) 
    filtered_dico_column = (df.isnull().sum()/np.shape(df)[0]*100).to_dict()
    filtered_dico_row = (df.isnull().sum(axis=1)/len(df.columns)*100).to_dict()
    return df, filtered_dico_column, filtered_dico_row

In [5]:
clean_list = []
row = []
column = []
for i in df_list:
    df, dico_column, dico_row = get_dico(i)
    clean_list.append(df)
    row.append(dico_row)
    column.append(dico_column)

In [6]:
predict = ["Comb Unrd Adj FE - Conventional Fuel"]
feature = []
for k in np.arange(3):
    L = []
    for i in column[k].keys():
        for j in ['FE', 'MPG', 'EPA', 'CO2', 'Guzzler', 'cost ', 'range', 'GHG', 'Smog', 'Desc']:
            if j in i:
                L.append(i)
    feature.append(list(set(column[k].keys())-set(L))+predict)

In [7]:
list_train = []
for i in np.arange(3):
    list_train.append(clean_list[i][feature[i]])  

In [8]:
row = []
column = []
df_list = []

for i in list_train:
    df, dico_column, dico_row = get_dico(i,60,100)
    row.append(dico_row)
    column.append(dico_column)
    df_list.append(df)

In [9]:
categorical = ['Mfr Name', 
               'Division', 
               'Carline', 
               'Carline Class',
               'Verify Mfr Cd',
               'Air Aspir Method',
               'Index (Model Type Index)', 
               'Transmission',
               'Trans',
               'Drive Sys', 
               'Fuel Usage  - Conventional Fuel',
               'Fuel Unit - Conventional Fuel',
               'Fuel Metering Sys Cd',
               'Car/Truck Category - Cash for Clunkers Bill.',
               'Oil Viscosity']

numerical = ['Model Year', 
             'Eng Displ', 
             '# Cyl', 
             'Intake Valves Per Cyl',
             '# Gears',
             'Annual Fuel1 Cost - Conventional Fuel',
             'Exhaust Valves Per Cyl',
             '$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) ',
             'Max Ethanol % - Gasoline',
             'Release Date']

boolean = ['Lockup Torque Converter',
           'Trans Creeper Gear', 
           'Cyl Deact?', 
           'Var Valve Timing?', 
           'Var Valve Lift?', 
           'Camless Valvetrain (Y or N)', 
           'Stop/Start System (Engine Management System) Code',
           'Suppressed?', 
           'Police/Emerg?',
           'Label Recalc?',
           'Unique Label?']

to_clean = ['Oil Viscosity']

predict = ["Comb Unrd Adj FE - Conventional Fuel"]

list_feature = categorical + predict + numerical + boolean 
list_feature_X = categorical + numerical + boolean 

all_categorical = categorical + boolean 
all_numerical = predict + numerical 

In [10]:
df_2018_clean, dico_column_test, dico_row_test = get_dico(clean_list[-1],101,100)

In [11]:
df_train = pd.concat(df_list, ignore_index=True)[list_feature]
df_test = df_2018_clean[list_feature]

In [12]:
def date_clean(date):
    date = str(date)
    if date != 'nan':
        date_convert = datetime.datetime.strptime(date, "%d/%m/%y").timestamp()
    else:
        date_convert = np.nan
    return date_convert

In [13]:
df_train['Release Date'] = df_train['Release Date'].apply(date_clean)
df_test['Release Date'] = df_test['Release Date'].apply(date_clean)

/Users/Louis/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Missing values

### Numerical

In [14]:
df_train_numerical = df_train[numerical]
df_test_numerical = df_test[numerical]

In [15]:
%%time
def fill_NaN(X_train, nbr_neighbors):
    distances = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i, x1 in enumerate(X_train):
        for j, x2 in enumerate(X_train):
            dist = (x1 - x2) ** 2
            nan_mask = np.isnan(dist)
            distances[i, j] = dist[~nan_mask].mean() * X_train.shape[1]

    neighbors = np.argsort(distances, axis=1)[:, 1:]
    n_neighbors = nbr_neighbors
    X_train_knn = X_train.copy()
    for feature in range(X_train.shape[1]):
        has_missing_value = np.isnan(X_train[:, feature])
        for row in np.where(has_missing_value)[0]:
            neighbor_features = X_train[neighbors[row], feature]
            non_nan_neighbors = neighbor_features[~np.isnan(neighbor_features)]
            X_train_knn[row, feature] = non_nan_neighbors[:n_neighbors].mean()
    
    return X_train_knn, distances

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [ ]:
def get_filled_df(df):
    df_categorical = df[all_categorical+predict]
    df_numerical = df[numerical]
    X_knn, distances = fill_NaN(df_numerical.as_matrix(), 3)
    df_numerical_filled = pd.DataFrame(X_knn, columns=numerical)
    df_full = pd.concat([df_numerical_filled, df_categorical],axis=1)
    return df_full

In [ ]:
%%time
df_train_full = get_filled_df(df_train)
df_test_full = get_filled_df(df_test)

### Scaling

In [ ]:
def get_scaled(df_train, df_test, sep=True):
    scaler = StandardScaler()
    if sep == True:
        scaler.fit(df_train[all_numerical])
        df_train_numerical = pd.DataFrame(scaler.transform(df_train[all_numerical]), columns=all_numerical)
        df_test_numerical = pd.DataFrame(scaler.transform(df_test[all_numerical]), columns=all_numerical)
        df_train_scaled = pd.concat([df_train_numerical, df_train[all_categorical]],axis=1)
        df_test_scaled = pd.concat([df_test_numerical, df_test[all_categorical]],axis=1)
    else:
        scaler.fit(df_train)
        df_train_scaled = pd.DataFrame(scaler.transform(df_train), columns=df_train.columns)
        df_test_scaled = pd.DataFrame(scaler.transform(df_test), columns=df_test.columns)
    return df_train_scaled, df_test_scaled

In [ ]:
def get_dummy(df_train, df_test):
    df_dummy = pd.get_dummies(df_train[list_feature_X])
    df_test_dummy = pd.get_dummies(df_test[list_feature_X])

    final_feature = list(set(df_dummy.columns) - (set(df_dummy.columns) - set(df_test_dummy.columns)))

    X_train = df_dummy[final_feature]
    #X_train.loc[:,'Bias'] = pd.Series(1, index=X_train.index)
    Y_train = df_train[predict]
    X_test = df_test_dummy[final_feature]
    #X_test.loc[:, 'Bias'] = pd.Series(1, index=X_test.index)
    Y_test = df_test[predict]
    
    return X_train, Y_train, X_test, Y_test

In [ ]:
## Getting the dummies and scaling afterwards
X_train, Y_train, X_test, Y_test = get_dummy(df_train_full, df_test_full)
X_train, X_test = get_scaled(X_train, X_test,sep=False)
Y_train, Y_test = get_scaled(Y_train, Y_test,sep=False)

## IID Variables

In [ ]:
n = np.shape(df_test)[0]
m = np.shape(df_train)[0]
df_all = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
def select_data(df, stop):
    n = np.shape(df)[0]
    ind = np.arange(n)
    shuffle(ind)
    return(df.iloc[ind][:stop].reset_index(drop = True), df.iloc[ind][stop:].reset_index(drop = True)) 

In [ ]:
df_train_iid, df_test_iid = select_data(df_all, stop=n)

In [ ]:
%%time
df_train_iid_full = get_filled_df(df_train_iid)
df_test_iid_full = get_filled_df(df_test_iid)

In [ ]:
X_train_iid, Y_train_iid, X_test_iid, Y_test_iid = get_dummy(df_train_iid_full, df_test_iid_full)

In [ ]:
X_train_iid, X_test_iid = get_scaled(X_train_iid, X_test_iid,sep=False)
Y_train_iid, Y_test_iid = get_scaled(Y_train_iid, Y_test_iid,sep=False)

## Finding the best linear model

### Ridge

In [ ]:
param_grid = {'alpha': np.logspace(-3, 3, 13)}
grid_ridge = GridSearchCV(Ridge(), param_grid, cv=10)
grid_ridge.fit(X_train, Y_train)
print(grid_ridge.best_params_)
print(grid_ridge.best_score_)
ridge = grid_ridge.best_estimator_

In [ ]:
plt.figure(figsize=(8,8))
plt.xscale('log')
plt.title('Effect of alpha on Ridge regression', fontsize=20)
plt.xlabel('alpha', fontsize=18)
plt.ylabel('validation score', fontsize=18)
plt.plot(param_grid['alpha'], grid_ridge.cv_results_['mean_test_score'], marker='o')
plt.fill_between(param_grid['alpha'], grid_ridge.cv_results_['mean_test_score'] 
                 + grid_ridge.cv_results_['std_test_score'], 
                 grid_ridge.cv_results_['mean_test_score'] - 
                 grid_ridge.cv_results_['std_test_score'], 
                 facecolor='blue', 
                 alpha=0.1)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Ridge coefficent', fontsize=18)
plt.xlabel('coefficient n°', fontsize=16)
plt.ylabel('value', fontsize=16)
plt.scatter(np.arange(len(ridge.coef_[0])), ridge.coef_, c=np.sign(ridge.coef_), cmap="bwr_r")
plt.show()

### Lasso

In [ ]:
%%time
param_grid = {'alpha': np.logspace(-3, 3, 13)}
grid_lasso = GridSearchCV(Lasso(), param_grid, cv=10)
grid_lasso.fit(X_train, Y_train)
print(grid_lasso.best_params_)
print(grid_lasso.best_score_)
lasso = grid_lasso.best_estimator_

In [ ]:
plt.figure(figsize=(12,8))
plt.xscale('log')
plt.title('Effect of alpha on Lasso regression', fontsize=20)
plt.xlabel('alpha', fontsize=18)
plt.ylabel('validation score', fontsize=18)
plt.plot(param_grid['alpha'], grid_lasso.cv_results_['mean_test_score'], marker='o')
plt.fill_between(param_grid['alpha'], grid_lasso.cv_results_['mean_test_score']
                 + grid_lasso.cv_results_['std_test_score'], 
                 grid_lasso.cv_results_['mean_test_score']
                 - grid_lasso.cv_results_['std_test_score'], 
                 facecolor='blue', 
                 alpha=0.1)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Lasso coefficent', fontsize=18)
plt.xlabel('coefficient n°', fontsize=16)
plt.ylabel('value', fontsize=16)
plt.scatter(np.arange(len(lasso.coef_)), lasso.coef_, c=np.sign(lasso.coef_), cmap="bwr_r")
plt.show()

### Elastic Net

In [ ]:
param_grid = {'alpha': np.logspace(-2, 3, 13)}
grid_elastic_net = GridSearchCV(ElasticNet(), param_grid, cv=10)
grid_elastic_net.fit(X_train, Y_train)
print(grid_elastic_net.best_params_)
print(grid_elastic_net.best_score_)
elastic_net = grid_elastic_net.best_estimator_

In [ ]:
plt.figure(figsize=(12,8))
plt.xscale('log')
plt.title('Effect of alpha on ElasticNet regression', fontsize=20)
plt.xlabel('alpha', fontsize=18)
plt.ylabel('validation score', fontsize=18)
plt.plot(param_grid['alpha'], grid_elastic_net.cv_results_['mean_test_score'], 
         marker='o')
plt.fill_between(param_grid['alpha'], grid_elastic_net.cv_results_['mean_test_score']
                 + grid_elastic_net.cv_results_['std_test_score'], 
                 grid_elastic_net.cv_results_['mean_test_score']
                 - grid_elastic_net.cv_results_['std_test_score'],
                 facecolor='blue', 
                 alpha=0.1)
plt.show()

### Evaluation of our best linear model, with and without the iid assumption

In [ ]:
def evaluate(best_param, X_train, X_test, assumption_iid = False, type='r'):
    if type == 'r':
        model_evaluate = Ridge(alpha=best_param)
    elif type == 'l':
        model_evaluate = Lasso(alpha=best_param)
    else:
        model_evaluate = ElasticNet(alpha=best_param)
        
    if assumption_iid == False:
        model_evaluate.fit(X_train, Y_train)
        score = model_evaluate.score(X_test, Y_test)
    else:
        model_evaluate.fit(X_train_iid, Y_train_iid)
        score = model_evaluate.score(X_test_iid, Y_test_iid)
   
    return score

In [ ]:
score = evaluate(grid_ridge.best_params_['alpha'], X_train, X_test)
print(score)
score_iid = evaluate(grid_ridge.best_params_['alpha'], X_train, X_test, assumption_iid = True)
print(score_iid)

## Task 2 : Feature engineering

### Getting polynomial features of numerical features

In [ ]:
poly = PolynomialFeatures()
categorical_dummies = list(set(X_train.columns) - set(X_train[numerical].columns))
X_train_categorical_poly = X_train[categorical_dummies]
X_train_numerical_poly = pd.DataFrame(poly.fit_transform(X_train[numerical]))
X_test_numerical_poly = pd.DataFrame(poly.transform(X_test[numerical]))
X_train_poly = pd.concat([X_train_numerical_poly, X_train[categorical_dummies]], axis=1)
X_test_poly = pd.concat([X_test_numerical_poly, X_test[categorical_dummies]], axis=1)

### Ridge

In [ ]:
param_grid_poly = {'alpha': np.logspace(-3, 3, 13)}
grid_ridge_poly = GridSearchCV(Ridge(), param_grid_poly, cv=10)
grid_ridge_poly.fit(X_train_poly, Y_train)
print(grid_ridge_poly.best_params_)
print(grid_ridge_poly.best_score_)
ridge_poly = grid_ridge_poly.best_estimator_

In [ ]:
plt.figure(figsize=(8,8))
plt.xscale('log')
plt.title('Effect of alpha on Ridge regression', fontsize=20)
plt.xlabel('alpha', fontsize=18)
plt.ylabel('validation score', fontsize=18)
plt.plot(param_grid['alpha'], grid_ridge_poly.cv_results_['mean_test_score'], marker='o')
plt.fill_between(param_grid['alpha'], grid_ridge_poly.cv_results_['mean_test_score'] 
                 + grid_ridge_poly.cv_results_['std_test_score'], 
                 grid_ridge_poly.cv_results_['mean_test_score'] - 
                 grid_ridge_poly.cv_results_['std_test_score'], 
                 facecolor='blue', 
                 alpha=0.1)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Ridge coefficent', fontsize=18)
plt.xlabel('coefficient n°', fontsize=16)
plt.ylabel('value', fontsize=16)
plt.scatter(np.arange(len(ridge_poly.coef_[0])), ridge_poly.coef_, c=np.sign(ridge_poly.coef_), cmap="bwr_r")
plt.show()

### Lasso

In [ ]:
%%time
param_grid_poly = {'alpha': np.logspace(-3, 3, 13)}
grid_lasso_poly = GridSearchCV(Lasso(normalize=True), param_grid_poly, cv=10)
grid_lasso_poly.fit(X_train_poly, Y_train)
print(grid_lasso_poly.best_params_)
print(grid_lasso_poly.best_score_)
lasso_poly = grid_lasso_poly.best_estimator_

In [ ]:
plt.figure(figsize=(12,8))
plt.xscale('log')
plt.title('Effect of alpha on Lasso regression', fontsize=20)
plt.xlabel('alpha', fontsize=18)
plt.ylabel('validation score', fontsize=18)
plt.plot(param_grid_poly['alpha'], grid_lasso_poly.cv_results_['mean_test_score'], marker='o')
plt.fill_between(param_grid_poly['alpha'], grid_lasso_poly.cv_results_['mean_test_score']
                 + grid_lasso_poly.cv_results_['std_test_score'], 
                 grid_lasso_poly.cv_results_['mean_test_score']
                 - grid_lasso_poly.cv_results_['std_test_score'], 
                 facecolor='blue', 
                 alpha=0.1)
plt.show()

## Elastic Net

In [ ]:
param_grid_poly = {'alpha': np.logspace(-1, 3, 13)}
grid_elastic_net_poly = GridSearchCV(ElasticNet(), param_grid_poly, cv=10)
grid_elastic_net_poly.fit(X_train_poly, Y_train)
print(grid_elastic_net_poly.best_params_)
print(grid_elastic_net_poly.best_score_)
elastic_net_poly = grid_elastic_net_poly.best_estimator_

In [ ]:
plt.figure(figsize=(12,8))
plt.xscale('log')
plt.title('Effect of alpha on ElasticNet regression', fontsize=20)
plt.xlabel('alpha', fontsize=18)
plt.ylabel('validation score', fontsize=18)
plt.plot(param_grid_poly['alpha'], grid_elastic_net_poly.cv_results_['mean_test_score'], 
         marker='o')
plt.fill_between(param_grid_poly['alpha'], grid_elastic_net_poly.cv_results_['mean_test_score']
                 + grid_elastic_net_poly.cv_results_['std_test_score'], 
                 grid_elastic_net_poly.cv_results_['mean_test_score']
                 - grid_elastic_net_poly.cv_results_['std_test_score'],
                 facecolor='blue', 
                 alpha=0.1)
plt.show()

## Task 3 : Any model

In [ ]:
## No scaling
X_train, Y_train, X_test, Y_test = get_dummy(df_train_full, df_test_full)

In [ ]:
## Getting polynomial
poly = PolynomialFeatures()
categorical_dummies = list(set(X_train.columns) - set(X_train[numerical].columns))
X_train_numerical_poly = pd.DataFrame(poly.fit_transform(X_train[numerical]))
X_test_numerical_poly = pd.DataFrame(poly.transform(X_test[numerical]))
X_train_poly = pd.concat([X_train_numerical_poly, X_train[categorical_dummies]], axis=1)
X_test_poly = pd.concat([X_test_numerical_poly, X_test[categorical_dummies]], axis=1)

In [ ]:
grad_model = XGBRegressor(n_estimators = 300)
m = int(np.shape(X_train)[0]/10)
X_train_val = X_train[:m]
Y_train_val = Y_train[:m]
X_train_tr = X_train[m:]
Y_train_tr = Y_train[m:]
grad_model.fit(X_train_tr, Y_train_tr)
print(grad_model.score(X_train_val, Y_train_val))

In [ ]:
final_model = XGBRegressor(n_estimators = 300)
m = int(np.shape(X_train)[0]/10)
X_train_poly_val = X_train_poly[:m]
Y_train_poly_val = Y_train[:m]
X_train_poly_tr = X_train_poly[m:]
Y_train_tr = Y_train[m:]
final_model.fit(X_train_poly_tr, Y_train_tr)
print(final_model.score(X_train_poly_val, Y_train_poly_val))

In [ ]:
poly = PolynomialFeatures()
categorical_dummies = list(set(X_train_iid.columns) - set(X_train_iid[numerical].columns))
X_train_iid_categorical_poly = X_train_iid[categorical_dummies]
X_train_iid_numerical_poly = pd.DataFrame(poly.fit_transform(X_train_iid[numerical]))
X_test_iid_numerical_poly = pd.DataFrame(poly.transform(X_test_iid[numerical]))
X_train_iid_poly = pd.concat([X_train_iid_numerical_poly, X_train_iid[categorical_dummies]], axis=1)
X_test_iid_poly = pd.concat([X_test_iid_numerical_poly, X_test_iid[categorical_dummies]], axis=1)

In [ ]:
final_model = XGBRegressor(n_estimators = 300)
final_model.fit(X_train_poly, Y_train)
print('Score without iid assumption :', final_model.score(X_test_poly, Y_test))

final_model_iid = XGBRegressor(n_estimators = 300)
final_model_iid.fit(X_train_iid_poly, Y_train_iid)
print('Score with iid assumption :', final_model_iid.score(X_test_iid_poly, Y_test_iid))

## Task 4 : Feature Selections

In [ ]:
L = []
dico = final_model.booster().get_score(importance_type='weight')
for key, value in dico.items():
    if key in str(np.arange(100)):
        key = int(key)
    L.append(key)

In [ ]:
X_train_filtered = X_train_poly[L]
X_test_filtered = X_test_poly[L]

In [ ]:
final_model = XGBRegressor(n_estimators = 300)
final_model.fit(X_train_filtered, Y_train)
print('Score without iid assumption :', final_model.score(X_test_filtered, Y_test))

In [ ]:
L_sparse = []
dico = final_model.booster().get_score(importance_type='weight')
for key, value in dico.items():
    if key in str(np.arange(100)):
        key = int(key)
    if value > 10:
        L_sparse.append(key)

In [ ]:
L_sparse

In [ ]:
X_train_sparse = X_train_poly[L_sparse]
X_test_sparse = X_test_poly[L_sparse]

In [ ]:
final_model = XGBRegressor(n_estimators = 300)
final_model.fit(X_train_sparse, Y_train)
print('Score without iid assumption :', final_model.score(X_test_sparse, Y_test))